
In this file i remove duplicate values and filled na values 

merged categoreies of some features

statistical tests 

one hot encoding

normalization





In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

In [ ]:
# reading the file
path = "/content/drive/MyDrive/adult.csv"
df = pd.read_csv(path)

In [ ]:
output_var = df['income']

we have checked in eda that we have duplicated rows present in our data so i am gonna drop them

In [ ]:
# checking duplicated data 

# check the shape
print(f"we have {df.shape[0]} rows and {df.shape[1]} columns")

duplicate_data = df.duplicated()
print(f"we have {duplicate_data.sum()} duplicate data")

# we have duplicate data,removing them from our data
df.drop_duplicates(inplace = True)

# check the shape again
print(f"we have {df.shape[0]} rows and {df.shape[1]} columns")

# checking duplicated data again
duplicate_data = df.duplicated()
print(f"we have {duplicate_data.sum()} duplicate data")

we have checked in eda that we have question mark in three columns which is workclass,occupation and native.country , so all these three are nominal categorical variable so i am gonna create new label for the missing values named by 'missing'

In [ ]:
# filling the question marks with missing label in data
df['workclass'].replace("?",'Missing',inplace = True)
df['occupation'].replace("?",'Missing',inplace = True)
df['native.country'].replace("?",'Missing',inplace = True)

In [ ]:
# saving csv file in pands with filled na values 
# df.to_csv('/content/drive/My Drive/adult_filled_na.csv',index=False)

In [ ]:
# rechecking the question mark in data
question_mark = "?"
question_mark = df.isin([question_mark]).sum()
print(question_mark)

**merging categories of some features**

In  Workclasss, merging Never-worked and Without- pay into one category which  is "Neverworked/withoutpay"

In race, merging Asian-Pac-Islander and Amer-Indian-Eskimo and other into "other_race"

In native.country, Except United states merging all ther countries into "other_countries"

Education and education.num are overlapping so we will delete education column.

education_dict = {"preschool" :1, "1st-4th": 2, "5th-6th": 3, "7th-8th": 4, "9th": 5, "10th": 6, "11th": 7, "12th": 8, "HS-grad": 9, "Some-college": 10, "Assoc-voc": 11, "Assoc-acdm": 12, "Bachelors": 13, "Masters": 14,"Prof-school": 15, "Doctorate": 16}

In education.num,merginng preschool to 12th into "Higher_secondary". 

In [ ]:
# merging Never-worked and Without- pay into one category which is "Neverworked/withoutpay"
df['workclass'].replace('Without-pay','Neverworked/withoutpay',inplace = True)
df['workclass'].replace('Never-worked','Neverworked/withoutpay',inplace = True)

#merging Asian-Pac-Islander and Amer-Indian-Eskimo and other into "other_race"
df['race'].replace('Asian-Pac-Islander','Other_race',inplace = True)
df['race'].replace('Amer-Indian-Eskimo','Other_race',inplace = True)
df['race'].replace('Other','Other_race',inplace = True)

# Except United states merging all ther countries into "other_country
df['native.country'].where(~(df['native.country'] !=  'United-States'), other = "Other_country",inplace = True)

#In education.num,merginng preschool to 12th into "Higher_secondary". 
df['education.num'].replace([1,2,3,4,5,6,7,8],1,inplace = True)
df['education.num'].replace(9,2,inplace = True)
df['education.num'].replace(10,3,inplace = True)
df['education.num'].replace(11,4,inplace = True)
df['education.num'].replace(12,5,inplace = True)
df['education.num'].replace(13,6,inplace = True)
df['education.num'].replace(14,7,inplace = True)
df['education.num'].replace(15,8,inplace = True)
df['education.num'].replace(16,9,inplace = True)

In [ ]:
#dropping some columns(education)
df.drop(['education'],axis = 1,inplace = True)

**Feature Selection by statistical test**


In [ ]:
from scipy.stats import ttest_ind 

def ttest(df,var1):
  cat1 = df.query("income == '>50K'")
  cat2 = df.query("income == '<=50K'")

  stat,p = ttest_ind(cat1[var1], cat2[var1])

   # interpret p-value
  alpha = 0.05
  print(f"\nalpha is {alpha} , p value is {p}")
  if p <= alpha:
	  print('\nreject null hypothesis,there is relationship between variables')
  else:
	  print('\nfail to reject null hypothesis,there is no relationship between variables')
   
  return

In [ ]:
# continuous_var = ['age','fnlwgt', 'capital.gain','capital.loss', 'hours.per.week']
ttest(df,'capital.loss')

age = p value is 0.0

fnlwgt = p value is 0.08652724317860144

capital.gain = p value is 0.0

capital.loss = p value is 4.022159409210232e-164

hours.per.week = p value is 0.0

By analyzing p values, Except fnlwgt i will take all variable for normalization

In [ ]:
# chi-squared test 

from scipy.stats import chi2_contingency
from scipy.stats import chi2

def chi_squared_test(df,var1,var2):
  df_table = pd.crosstab(df[var1],df[var2])
  print(df_table)
  
  stat, p, dof, expected = chi2_contingency(df_table)
  print("\nthe degree of freedom is :",dof)
  print("\nthe expected value is \n",expected)
  
  # interpret p-value
  alpha = 0.05
  print(f"\nalpha is {alpha} , p value is {p}")
  if p <= alpha:
	  print('\nreject null hypothesis,there is relationship between variables')
  else:
	  print('\nfail to reject null hypothesis,there is no relationship between variables')
   
  return

In [ ]:
#categ_var = ['sex','workclass','marital.status','occupation','relationship','race','native.country','education.num']

chi_squared_test(df,'income','education.num')

sex = p value is 0.0

workclass = p value is 2.6455448138696704e-221

marital.status = p value is 0.0

occupation = p value is 0.0

relationship = p value is 0.0

race = p value is 5.261106981519217e-61

native.country = p value is 7.804143463331444e-10

education.num = p value is 0.0

**one Hot Enoding**

In [ ]:
#categorical_features = ['workclass','marital.status','occupation','relationship','race','sex','native.country']
df = pd.get_dummies(df,columns=['workclass','marital.status','occupation','relationship','race','sex','native.country'],drop_first=True)

In [ ]:
# splitting data 
X = df.drop('income',axis=1)
Y= df.income

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y = le.transform(Y)
Y_mappings = {index: label for index, label in enumerate(le.classes_)}
Y_mappings

{0: '<=50K', 1: '>50K'}

**Normalization**

In [ ]:
# normalizing the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X),columns = X.columns)

In [ ]:
# doing train test split and saving into dataframe

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)
X_train_adult = pd.DataFrame(X_train)
X_test_adult = pd.DataFrame(X_test)
Y_train_adult = pd.DataFrame(Y_train)
Y_test_adult = pd.DataFrame(Y_test)

In [ ]:
# checking the shape of the data
print("shape of original dataset :", df.shape)
print("shape of X_train", X_train.shape)
print("shape of Y_train", Y_train.shape)
print("shape of X_test", X_test.shape)
print("shape of Y_test]", Y_test.shape)

In [ ]:
# saving csv file
X_train_adult.to_csv('/content/drive/My Drive/X_train_adult.csv',index=False)
X_test_adult.to_csv('/content/drive/My Drive/X_test_adult.csv',index=False)
Y_train_adult.to_csv('/content/drive/My Drive/Y_train_adult.csv',index=False)
Y_test_adult.to_csv('/content/drive/My Drive/Y_test_adult.csv',index=False)